In [ ]:
# This part uses visualization to explain the EfficientNet model

In [ ]:
!pip install timm
!pip install optuna
!pip install pytorch-gradcam
!pip install captum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 47.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch-gradcam: filename=pytorch_gradcam-0.2.1-py3-none-any.whl size=5248 sha256=bc61ba5042da4cb15ca33df3b8fca217998edecb8d1dd104c57390dd6697d19a
  Stored in directory: /root/.cache/pip/wheels/6f/f1/8f/96c81d13f617841f23cae192a77fea3e9e988d058ba9414f2c
Successfully built pytorch-gradcam
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.6 MB/s eta 0:00:00


In [ ]:
# Use EfficientNet(tf_efficientnet_b5_ns) draw the curves with best hyperparameters
import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from google.colab import drive
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import time
import copy
from torch.utils.data import Dataset
from torchvision.io import read_image
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, confusion_matrix
import seaborn as sns
import random
import timm
from gradcam.utils import visualize_cam
from gradcam import GradCAM, GradCAMpp

# Connect to the google conlab
drive.mount('/content/drive')

# Use transforms to do data augmentation and preprocess
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomRotation(45),  # Rotate randomly between 0 and 45 degrees
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Randomly crop and scale to 224x224 size
    transforms.Grayscale(num_output_channels=3),  # Grayscale
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalization
])

class Knots_Dataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.data = []
        self.labels = []
        self.label_encoder = LabelEncoder()

        # Loop over all conditions
        for knot_type in os.listdir(img_dir):
            for light_condition in os.listdir(os.path.join(img_dir, knot_type)):
                for tension_condition in os.listdir(os.path.join(img_dir, knot_type, light_condition)):
                    images = os.listdir(os.path.join(img_dir, knot_type, light_condition, tension_condition))
                    random.shuffle(images)  # Shuffle the images
                    for image in images:
                        self.data.append(os.path.join(img_dir, knot_type, light_condition, tension_condition, image))
                        self.labels.append(knot_type)

        # Fit the label encoder and transform the labels
        self.labels = self.label_encoder.fit_transform(self.labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx]
        image = Image.open(img_path)
        image = image.convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

# Create a custom data set
dataset = Knots_Dataset("/content/drive/My Drive/MSc Project/rawdata", transform=data_transforms)

# Split the dataset into training set and validation set
train_dataset = []
val_dataset = []
for i in range(len(dataset)):
    if i % 4 == 0:  # Put 25% of the images in the validation set
        val_dataset.append(dataset[i])
    else:  # Put the rest in the training set
        train_dataset.append(dataset[i])

# Create a Dataloader
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

dataloaders = {'train': train_dataloader, 'val': val_dataloader}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

# Check if model can be trained on GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Runing on GPU")
    print("CUDA device count: ", torch.cuda.device_count())
else:
    device = torch.device("cpu")
    print("Runing on CPU")

# Save the model results
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []
train_f1_scores, val_f1_scores = [], []

def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_preds = None
    best_labels = None

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            running_preds = []
            running_labels = []

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                # zero(clean) the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                running_preds.extend(preds.cpu().numpy())
                running_labels.extend(labels.cpu().numpy())

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            epoch_acc = epoch_acc.cpu().numpy()  # Move tensor to CPU and convert to numpy(debug)
            epoch_f1_score = f1_score(running_labels, running_preds, average='macro')
            epoch_f1_score = torch.tensor(epoch_f1_score).cpu().numpy()  # Move tensor to CPU and convert to numpy(debug)

            if phase == 'train':
                train_losses.append(epoch_loss)
                train_accuracies.append(epoch_acc)
                train_f1_scores.append(epoch_f1_score)
            else:
                val_losses.append(epoch_loss)
                val_accuracies.append(epoch_acc)
                val_f1_scores.append(epoch_f1_score)

            print('{} Loss: {:.5f}  Acc: {:.5f}   F1_Score: {:.5f}'.format(
                phase, epoch_loss, epoch_acc, epoch_f1_score))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                best_preds = running_preds
                best_labels = running_labels

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:5f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)

    return model, best_preds, best_labels



# [I 2023-07-18 19:35:10,874] Trial 49 finished with value: 0.9851851851851853
# and parameters: {'lr': 0.00015305939437836783, 'optimizer': 'RMSprop', 'weight_decay': 2.0510689201040282e-10}. Best is trial 49 with value: 0.9851851851851853.
# Use the best hyperparameters to train the model
best_lr = 0.00001504939437836783  # 0.00001104939437836783
# 0.00001705939437836783 Best val Acc: 0.955556
# 0.00001103939437836783 Best val Acc: 0.955556
# 0.00001104939437836783 Best val Acc: 0.959259
# 0.00001104939637836783 Best val Acc: 0.970370
best_optimizer_name = 'RMSprop'
best_weight_decay = 2.0510689201040282e-10

# Load a pretrained model(EfficientNet)
model = timm.create_model('tf_efficientnet_b5_ns', pretrained=True)
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 5)  # Change the classifier layer to 5
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Use the best optimizer
if best_optimizer_name == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=best_lr, weight_decay=best_weight_decay)
elif best_optimizer_name == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=best_lr, weight_decay=best_weight_decay)
else:
    optimizer = optim.RMSprop(model.parameters(), lr=best_lr, weight_decay=best_weight_decay)


# Train and evaluate the best model
model, best_preds, best_labels = train_model(model, criterion, optimizer, num_epochs=25)








# Define a function to apply GradCAM to a given image
def apply_gradcam(img, label):
    # Add batch dimension and move to device
    img = img.unsqueeze(0).to(device)

    # Set the model to evaluation mode and get the output
    model.eval()
    out = model(img)

    # Initialize GradCAM
    gradcam = GradCAM(model, model.conv_head)

    # Get the GradCAM heatmap
    mask, _ = gradcam(img)
    heatmap, result = visualize_cam(mask, img)

    # Plot original image and heatmap
    fig, axarr = plt.subplots(nrows=1, ncols=2)
    img = img.squeeze().cpu().numpy().transpose(1, 2, 0)  # Squeeze before transpose
    img = (img - img.min()) / (img.max() - img.min())
    axarr[0].imshow(img)
    heatmap = heatmap.squeeze().cpu().numpy().transpose(1, 2, 0)  # Squeeze and transpose before displaying
    axarr[1].imshow(heatmap)
    plt.show()





# Loop over all conditions
for knot_type in os.listdir("/content/drive/My Drive/MSc Project/rawdata"):
    for light_condition in os.listdir(os.path.join("/content/drive/My Drive/MSc Project/rawdata", knot_type)):
        for tension_condition in os.listdir(os.path.join("/content/drive/My Drive/MSc Project/rawdata", knot_type, light_condition)):
            # Get the first image of this condition
            image_file = os.listdir(os.path.join("/content/drive/My Drive/MSc Project/rawdata", knot_type, light_condition, tension_condition))[0]
            img_path = os.path.join("/content/drive/My Drive/MSc Project/rawdata", knot_type, light_condition, tension_condition, image_file)
            image = Image.open(img_path)
            image = image.convert('RGB')
            image = data_transforms(image)
            label = knot_type

            print(f"Applying GradCAM for knot type {knot_type}, light condition {light_condition}, tension condition {tension_condition}")
            apply_gradcam(image, label)













# Define knot types
knot_types = ['Bowline', 'Clove_Hitch', 'Figure_8_Knot', 'Overhand', 'Reef']

# Plot confusion matrix
confusion_mtx = confusion_matrix(best_labels, best_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues', xticklabels=knot_types, yticklabels=knot_types)
plt.xlabel('Prediction')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()



# Plot loss curves
plt.figure()
plt.plot(train_losses, label='Train')
plt.plot(val_losses, label='Validation')
plt.title('Loss Curves')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot accuracy curves
plt.figure()
plt.plot(train_accuracies, label='Train')
plt.plot(val_accuracies, label='Validation')
plt.title('Accuracy Curves')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot F1 Score curves
plt.figure()
plt.plot(train_f1_scores, label='Train')
plt.plot(val_f1_scores, label='Validation')
plt.title('F1 Score Curves')
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.legend()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Use EfficientNet(tf_efficientnet_b5_ns) draw the curves with best hyperparameters
import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from google.colab import drive
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import time
import copy
from torch.utils.data import Dataset
from torchvision.io import read_image
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, confusion_matrix
import seaborn as sns
import random
import timm
from gradcam.utils import visualize_cam
from gradcam import GradCAM, GradCAMpp
from captum.attr import IntegratedGradients
from captum.attr import visualization as viz
from captum.attr import Occlusion

# Connect to the google conlab
drive.mount('/content/drive')

default_cmap = plt.cm.viridis

# Use transforms to do data augmentation and preprocess
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomRotation(45),  # Rotate randomly between 0 and 45 degrees
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Randomly crop and scale to 224x224 size
    transforms.Grayscale(num_output_channels=3),  # Grayscale
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalization
])

class Knots_Dataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.data = []
        self.labels = []
        self.label_encoder = LabelEncoder()

        # Loop over all conditions
        for knot_type in os.listdir(img_dir):
            for light_condition in os.listdir(os.path.join(img_dir, knot_type)):
                for tension_condition in os.listdir(os.path.join(img_dir, knot_type, light_condition)):
                    images = os.listdir(os.path.join(img_dir, knot_type, light_condition, tension_condition))
                    random.shuffle(images)  # Shuffle the images
                    for image in images:
                        self.data.append(os.path.join(img_dir, knot_type, light_condition, tension_condition, image))
                        self.labels.append(knot_type)

        # Fit the label encoder and transform the labels
        self.labels = self.label_encoder.fit_transform(self.labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx]
        image = Image.open(img_path)
        image = image.convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

    def get_label_int(self, label_str):
        return self.label_encoder.transform([label_str])[0]

# Create a custom data set
dataset = Knots_Dataset("/content/drive/My Drive/MSc Project/rawdata", transform=data_transforms)

# Split the dataset into training set and validation set
train_dataset = []
val_dataset = []
for i in range(len(dataset)):
    if i % 4 == 0:  # Put 25% of the images in the validation set
        val_dataset.append(dataset[i])
    else:  # Put the rest in the training set
        train_dataset.append(dataset[i])

# Create a Dataloader
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

dataloaders = {'train': train_dataloader, 'val': val_dataloader}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

# Check if model can be trained on GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Runing on GPU")
    print("CUDA device count: ", torch.cuda.device_count())
else:
    device = torch.device("cpu")
    print("Runing on CPU")

# Save the model results
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []
train_f1_scores, val_f1_scores = [], []

def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_preds = None
    best_labels = None

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            running_preds = []
            running_labels = []

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                # zero(clean) the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                running_preds.extend(preds.cpu().numpy())
                running_labels.extend(labels.cpu().numpy())

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            epoch_acc = epoch_acc.cpu().numpy()  # Move tensor to CPU and convert to numpy(debug)
            epoch_f1_score = f1_score(running_labels, running_preds, average='macro')
            epoch_f1_score = torch.tensor(epoch_f1_score).cpu().numpy()  # Move tensor to CPU and convert to numpy(debug)

            if phase == 'train':
                train_losses.append(epoch_loss)
                train_accuracies.append(epoch_acc)
                train_f1_scores.append(epoch_f1_score)
            else:
                val_losses.append(epoch_loss)
                val_accuracies.append(epoch_acc)
                val_f1_scores.append(epoch_f1_score)

            print('{} Loss: {:.5f}  Acc: {:.5f}   F1_Score: {:.5f}'.format(
                phase, epoch_loss, epoch_acc, epoch_f1_score))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                best_preds = running_preds
                best_labels = running_labels

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:5f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)

    return model, best_preds, best_labels



# [I 2023-07-18 19:35:10,874] Trial 49 finished with value: 0.9851851851851853
# and parameters: {'lr': 0.00015305939437836783, 'optimizer': 'RMSprop', 'weight_decay': 2.0510689201040282e-10}. Best is trial 49 with value: 0.9851851851851853.
# Use the best hyperparameters to train the model
best_lr = 0.00001504939437836783  # 0.00001104939437836783
# 0.00001705939437836783 Best val Acc: 0.955556
# 0.00001103939437836783 Best val Acc: 0.955556
# 0.00001104939437836783 Best val Acc: 0.959259
# 0.00001104939637836783 Best val Acc: 0.970370
best_optimizer_name = 'RMSprop'
best_weight_decay = 2.0510689201040282e-10

# Load a pretrained model(EfficientNet)
model = timm.create_model('tf_efficientnet_b5_ns', pretrained=True)
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 5)  # Change the classifier layer to 5
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Use the best optimizer
if best_optimizer_name == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=best_lr, weight_decay=best_weight_decay)
elif best_optimizer_name == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=best_lr, weight_decay=best_weight_decay)
else:
    optimizer = optim.RMSprop(model.parameters(), lr=best_lr, weight_decay=best_weight_decay)


# Train and evaluate the best model
model, best_preds, best_labels = train_model(model, criterion, optimizer, num_epochs=25)









# Define a function to apply Integrated Gradients to a given image
def apply_integrated_gradients(img, label):
    # Convert label from string to int
    label_to_int = {'Bowline': 0, 'Clove_Hitch': 1, 'Figure_8_Knot': 2, 'Overhand': 3, 'Reef': 4}
    label = label_to_int[label]

    # Add batch dimension and move to device
    img = img.unsqueeze(0).to(device)

    # Set the model to evaluation mode and get the output
    model.eval()
    out = model(img)

    # Initialize IntegratedGradients object
    ig = IntegratedGradients(model)

    # Compute attribution scores
    attributions_ig = ig.attribute(img, target=label)

    # Plot original image and heatmap
    fig, axarr = plt.subplots(nrows=1, ncols=2)
    img = img.squeeze().cpu().numpy().transpose(1, 2, 0)  # Squeeze before transpose
    img = (img - img.min()) / (img.max() - img.min())
    axarr[0].imshow(img)
    attributions_ig = attributions_ig.squeeze().cpu().numpy().transpose(1, 2, 0)
    attributions_ig = (attributions_ig - attributions_ig.min()) / (attributions_ig.max() - attributions_ig.min())
    axarr[1].imshow(attributions_ig)
    plt.show()


# Loop over all conditions
for knot_type in os.listdir("/content/drive/My Drive/MSc Project/rawdata"):
    for light_condition in os.listdir(os.path.join("/content/drive/My Drive/MSc Project/rawdata", knot_type)):
        for tension_condition in os.listdir(os.path.join("/content/drive/My Drive/MSc Project/rawdata", knot_type, light_condition)):
            # Get the first image of this condition
            image_file = os.listdir(os.path.join("/content/drive/My Drive/MSc Project/rawdata", knot_type, light_condition, tension_condition))[0]
            img_path = os.path.join("/content/drive/My Drive/MSc Project/rawdata", knot_type, light_condition, tension_condition, image_file)
            image = Image.open(img_path)
            image = image.convert('RGB')
            image = data_transforms(image)
            label = knot_type

            print(f"Applying Integrated Gradients for knot type {knot_type}, light condition {light_condition}, tension condition {tension_condition}")
            apply_integrated_gradients(image, label)













# Define knot types
knot_types = ['Bowline', 'Clove_Hitch', 'Figure_8_Knot', 'Overhand', 'Reef']

# Plot confusion matrix
confusion_mtx = confusion_matrix(best_labels, best_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues', xticklabels=knot_types, yticklabels=knot_types)
plt.xlabel('Prediction')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()



# Plot loss curves
plt.figure()
plt.plot(train_losses, label='Train')
plt.plot(val_losses, label='Validation')
plt.title('Loss Curves')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot accuracy curves
plt.figure()
plt.plot(train_accuracies, label='Train')
plt.plot(val_accuracies, label='Validation')
plt.title('Accuracy Curves')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot F1 Score curves
plt.figure()
plt.plot(train_f1_scores, label='Train')
plt.plot(val_f1_scores, label='Validation')
plt.title('F1 Score Curves')
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.legend()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Visualize the wrong label

# Use EfficientNet(tf_efficientnet_b5_ns) draw the curves with best hyperparameters
import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from google.colab import drive
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import time
import copy
from torch.utils.data import Dataset
from torchvision.io import read_image
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, confusion_matrix
import seaborn as sns
import random
import timm
from gradcam.utils import visualize_cam
from gradcam import GradCAM, GradCAMpp
from captum.attr import IntegratedGradients


# Connect to the google conlab
drive.mount('/content/drive')

default_cmap = plt.cm.viridis

# Use transforms to do data augmentation and preprocess
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomRotation(45),  # Rotate randomly between 0 and 45 degrees
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Randomly crop and scale to 224x224 size
    transforms.Grayscale(num_output_channels=3),  # Grayscale
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalization
])

class Knots_Dataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.data = []
        self.labels = []
        self.label_encoder = LabelEncoder()

        # Loop over all conditions
        for knot_type in os.listdir(img_dir):
            for light_condition in os.listdir(os.path.join(img_dir, knot_type)):
                for tension_condition in os.listdir(os.path.join(img_dir, knot_type, light_condition)):
                    images = os.listdir(os.path.join(img_dir, knot_type, light_condition, tension_condition))
                    random.shuffle(images)  # Shuffle the images
                    for image in images:
                        self.data.append(os.path.join(img_dir, knot_type, light_condition, tension_condition, image))
                        self.labels.append(knot_type)

        # Fit the label encoder and transform the labels
        self.labels = self.label_encoder.fit_transform(self.labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx]
        image = Image.open(img_path)
        image = image.convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    def get_image_path(self, idx):
        return self.data[idx]


# Create a custom data set
dataset = Knots_Dataset("/content/drive/My Drive/MSc Project/rawdata", transform=data_transforms)

# Split the dataset into training set and validation set
train_dataset = []
val_dataset = []
for i in range(len(dataset)):
    if i % 4 == 0:  # Put 25% of the images in the validation set
        val_dataset.append(dataset[i])
    else:  # Put the rest in the training set
        train_dataset.append(dataset[i])

# Create a Dataloader
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

dataloaders = {'train': train_dataloader, 'val': val_dataloader}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

# Check if model can be trained on GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Runing on GPU")
    print("CUDA device count: ", torch.cuda.device_count())
else:
    device = torch.device("cpu")
    print("Runing on CPU")

# Save the model results
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []
train_f1_scores, val_f1_scores = [], []

def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_preds = None
    best_labels = None

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            running_preds = []
            running_labels = []

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                # zero(clean) the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                running_preds.extend(preds.cpu().numpy())
                running_labels.extend(labels.cpu().numpy())

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            epoch_acc = epoch_acc.cpu().numpy()  # Move tensor to CPU and convert to numpy(debug)
            epoch_f1_score = f1_score(running_labels, running_preds, average='macro')
            epoch_f1_score = torch.tensor(epoch_f1_score).cpu().numpy()  # Move tensor to CPU and convert to numpy(debug)

            if phase == 'train':
                train_losses.append(epoch_loss)
                train_accuracies.append(epoch_acc)
                train_f1_scores.append(epoch_f1_score)
            else:
                val_losses.append(epoch_loss)
                val_accuracies.append(epoch_acc)
                val_f1_scores.append(epoch_f1_score)

            print('{} Loss: {:.5f}  Acc: {:.5f}   F1_Score: {:.5f}'.format(
                phase, epoch_loss, epoch_acc, epoch_f1_score))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                best_preds = running_preds
                best_labels = running_labels

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:5f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)

    return model, best_preds, best_labels



# [I 2023-07-18 19:35:10,874] Trial 49 finished with value: 0.9851851851851853
# and parameters: {'lr': 0.00015305939437836783, 'optimizer': 'RMSprop', 'weight_decay': 2.0510689201040282e-10}. Best is trial 49 with value: 0.9851851851851853.
# Use the best hyperparameters to train the model
best_lr = 0.00001504939437836783  # 0.00001104939437836783
# 0.00001705939437836783 Best val Acc: 0.955556
# 0.00001103939437836783 Best val Acc: 0.955556
# 0.00001104939437836783 Best val Acc: 0.959259
# 0.00001104939637836783 Best val Acc: 0.970370
best_optimizer_name = 'RMSprop'
best_weight_decay = 2.0510689201040282e-10

# Load a pretrained model(EfficientNet)
model = timm.create_model('tf_efficientnet_b5_ns', pretrained=True)
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 5)  # Change the classifier layer to 5
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Use the best optimizer
if best_optimizer_name == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=best_lr, weight_decay=best_weight_decay)
elif best_optimizer_name == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=best_lr, weight_decay=best_weight_decay)
else:
    optimizer = optim.RMSprop(model.parameters(), lr=best_lr, weight_decay=best_weight_decay)


# Train and evaluate the best model
model, best_preds, best_labels = train_model(model, criterion, optimizer, num_epochs=25)









# Define a function to apply Integrated Gradients to a given image
def apply_integrated_gradients(img, label):


    # Add batch dimension and move to device
    img = img.unsqueeze(0).to(device)

    # Set the model to evaluation mode and get the output
    model.eval()
    out = model(img)

    # Initialize IntegratedGradients object
    ig = IntegratedGradients(model)

    # Compute attribution scores
    attributions_ig = ig.attribute(img, target=label)

    # Plot original image and heatmap
    fig, axarr = plt.subplots(nrows=1, ncols=2)
    img = img.squeeze().cpu().numpy().transpose(1, 2, 0)  # Squeeze before transpose
    img = (img - img.min()) / (img.max() - img.min())
    axarr[0].imshow(img)
    attributions_ig = attributions_ig.squeeze().cpu().numpy().transpose(1, 2, 0)
    attributions_ig = (attributions_ig - attributions_ig.min()) / (attributions_ig.max() - attributions_ig.min())
    axarr[1].imshow(attributions_ig)
    plt.show()

# Define a function to apply GradCAM to a given image
def apply_gradcam(img, label):
    # Add batch dimension and move to device
    img = img.unsqueeze(0).to(device)

    # Set the model to evaluation mode and get the output
    model.eval()
    out = model(img)

    # Initialize GradCAM
    gradcam = GradCAM(model, model.conv_head)

    # Get the GradCAM heatmap
    mask, _ = gradcam(img)
    heatmap, result = visualize_cam(mask, img)

    # Plot original image and heatmap
    fig, axarr = plt.subplots(nrows=1, ncols=2)
    img = img.squeeze().cpu().numpy().transpose(1, 2, 0)  # Squeeze before transpose
    img = (img - img.min()) / (img.max() - img.min())
    axarr[0].imshow(img)
    heatmap = heatmap.squeeze().cpu().numpy().transpose(1, 2, 0)  # Squeeze and transpose before displaying
    axarr[1].imshow(heatmap)
    plt.show()


def visualize_misclassified_images(model, dataloader, max_images):
    misclassified_images = 0
    # 定义标签映射
    label_mapping = {0: 'Bowline', 1: 'Clove_Hitch', 2: 'Figure_8_Knot', 3: 'Overhand', 4: 'Reef'}

    # Go through the data
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Set the model to evaluation mode and get the output
        model.eval()
        outputs = model(inputs)

        # Get the predictions
        _, preds = torch.max(outputs, 1)

        # Find misclassified images
        misclassified_mask = preds != labels
        misclassified_inputs = inputs[misclassified_mask]
        misclassified_labels = labels[misclassified_mask]

        # Track misclassified images in current batch
        misclassified_images_batch = 0

        for img, label in zip(misclassified_inputs, misclassified_labels):
            print(f"Misclassified image {misclassified_images + 1}:")
            # 使用映射将标签数字转换为字符串
            print(f"True label: {label_mapping[label.item()]}")
            print(f"Predicted label: {label_mapping[preds[misclassified_mask][misclassified_images_batch].item()]}")

            # Apply Integrated Gradients and visualize
            print("Integrated Gradients visualization:")
            apply_integrated_gradients(img, label.item())

            # Apply GradCAM and visualize
            print("GradCAM visualization:")
            apply_gradcam(img, label.item())

            misclassified_images += 1
            misclassified_images_batch += 1

            # Stop if we have visualized enough images
            if misclassified_images >= max_images:
                return



# Apply the function to our data
visualize_misclassified_images(model, val_dataloader, max_images=100)


Output hidden; open in https://colab.research.google.com to view.